In [1]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0) 

import pandas as pd
import random
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

import os

import numpy as np
import torch.nn.functional as F

from google.colab import drive
drive.mount('/content/drive')

from sklearn.metrics import accuracy_score
from datetime import datetime
from IPython.display import clear_output 

Mounted at /content/drive


In [2]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.12.1+cu113
     |████████████████████████████████| 7.9 MB 15.8 MB/s 
     |████████████████████████████████| 3.5 MB 14.7 MB/s 
     |████████████████████████████████| 2.4 MB 14.5 MB/s 


In [3]:
train_dataset=[]
graphlist=[]
import networkx as nx
import torch_geometric
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader
#from torch.utils.data import DataLoader
from torch_geometric.utils import degree
from torch_geometric.utils import erdos_renyi_graph, to_networkx, from_networkx


%cd /content/drive/MyDrive/ADNI2/Controls
for i in range(1, 110):
    if os.path.isfile('{0:0>3}'.format(i)+'.txt'):
      tmp=pd.read_csv('{0:0>3}'.format(i)+'.txt', header=None)
      tmp=(tmp.to_numpy())
      tmp=torch.from_numpy(tmp)  
      tmp[tmp<=0.4]=0
      #print([tmp.type(torch.FloatTensor),torch.ones(1).type(torch.long)])
      G = nx.from_numpy_matrix(np.array(tmp))
      centrality = nx.eigenvector_centrality(G)
      x = []
      for i in range(len(centrality)):
        x.append([centrality[i]])
      x = np.array(x).reshape(90,1)
      data = from_networkx(G)
      #x = torch.tensor([[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1] ], dtype=torch.float)
      data.x = torch.FloatTensor(x)
      data.y = torch.tensor([1])
      graphlist.append(data)
      #graphlist.append([tmp.type(torch.FloatTensor),torch.ones(1).type(torch.long)])

%cd /content/drive/MyDrive/ADNI2/Patients
for i in range(1, 110):
    if os.path.isfile('{0:0>3}'.format(i)+'.txt'):
      tmp=pd.read_csv('{0:0>3}'.format(i)+'.txt', header=None)
      tmp=(tmp.to_numpy())
      tmp=torch.from_numpy(tmp) 
      tmp[tmp<=0.4]=0
      #print([tmp.type(torch.FloatTensor),torch.ones(1).type(torch.long)])
      G = nx.from_numpy_matrix(np.array(tmp))
      centrality = nx.eigenvector_centrality(G)
      x = []
      for i in range(len(centrality)):
        x.append([centrality[i]])
      x = np.array(x).reshape(90,1)
      data = from_networkx(G)
      #x = torch.tensor([[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1] ], dtype=torch.float)
      data.x = torch.FloatTensor(x)
      data.y = torch.tensor([0])
      graphlist.append(data)
      #graphlist.append([data, torch.zeros(1).type(torch.long)])

      #graphlist.append([tmp.type(torch.FloatTensor),torch.zeros(1).type(torch.long)])

/content/drive/.shortcut-targets-by-id/1n9P-S3rsi_g9yeb5KW1h7v-Bte7Cy43G/ADNI2/Controls
/content/drive/.shortcut-targets-by-id/1n9P-S3rsi_g9yeb5KW1h7v-Bte7Cy43G/ADNI2/Patients


In [4]:
dataset = random.sample(graphlist, len(graphlist))
train_dataset = dataset[:150]
test_dataset = dataset[150:]
train_loader = DataLoader(train_dataset, batch_size=25, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=25, shuffle=False)

In [5]:
from torch_geometric.nn import GATConv, GraphConv
from torch_geometric.nn.conv.gatv2_conv import GATv2Conv
from torch_geometric.nn import global_mean_pool, global_sort_pool

class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()

        self.in_head = 8
        self.out_head = 1
        self.conv1 = GraphConv(in_channels=1, out_channels=16)
        self.conv2 = GraphConv(in_channels=16, out_channels=32)
        self.conv3 = GraphConv(in_channels=32, out_channels=32)
        self.lin1 = torch.nn.Linear(32, 16)
        self.lin2 = torch.nn.Linear(16, 1)
        self.sig = torch.nn.Sigmoid()

    def forward(self, data):
        x, edge_index, weight, batch = data.x, data.edge_index, data.weight, data.batch 
        x = self.conv1(x, edge_index, weight)
        x = F.relu(x)
        #print(x.size())
        x = self.conv2(x, edge_index, weight)
        x = F.relu(x)
        #print(x.size())
        x = self.conv3(x, edge_index, weight)
        x = F.relu(x)
        #print(x.size())
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        #x = F.dropout(x, p=0.2, training=self.training)
        x = self.lin1(x)
        x = self.lin2(x)
        #print(x)
        
        return x
        #return F.log_softmax(x, dim=1)

In [6]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNN().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7649
Epoch: 002, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.7369
Epoch: 003, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6880
Epoch: 004, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7232
Epoch: 005, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6953
Epoch: 006, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6964
Epoch: 007, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6843
Epoch: 008, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6994
Epoch: 009, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6976
Epoch: 010, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6864
Epoch: 011, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6878
Epoch: 012, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6894
Epoch: 013, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.7065
Epoch: 014, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6922
Epoch: 015, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6872
Epoch: 016, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6961
Epoch: 0

In [7]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import GCNConv, JumpingKnowledge, global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.num_layers  = 3
        self.hidden = 16
        self.conv1 = GCNConv(1, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(GCNConv(self.hidden, self.hidden))
        self.lin1 = Linear(self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index,weight, batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index, weight))
        x = global_mean_pool(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [8]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.7099
Epoch: 002, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6951
Epoch: 003, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6862
Epoch: 004, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6853
Epoch: 005, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6926
Epoch: 006, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6870
Epoch: 007, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6979
Epoch: 008, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7020
Epoch: 009, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6923
Epoch: 010, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6957
Epoch: 011, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7120
Epoch: 012, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6804
Epoch: 013, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7061
Epoch: 014, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6828
Epoch: 015, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6863
Epoch: 016, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6923
Epoch: 0

In [9]:
from torch.nn import Linear
from torch_geometric.nn import GINConv, JumpingKnowledge, global_mean_pool
from torch.nn import BatchNorm1d as BN
from torch.nn import Linear, ReLU, Sequential
#from torch_geometric.nn import JumpingKnowledge, SAGEConv, global_mean_pool

class GIN(torch.nn.Module):
    def __init__(self):
        super(GIN, self).__init__()
        self.conv1 = GINConv(
            Sequential(
                Linear(1, 16),
                ReLU(),
                Linear(16, 16),
                ReLU(),
                BN(16),
            ), train_eps=False)
        self.conv2 = GINConv(
            Sequential(
                Linear(16, 32),
                ReLU(),
                Linear(32, 32),
                ReLU(),
                BN(32),
            ), train_eps=False)
        self.conv3 = GINConv(
            Sequential(
                Linear(32, 32),
                ReLU(),
                Linear(32, 32),
                ReLU(),
                BN(32),
            ), train_eps=False)
        self.lin1 = Linear(32, 16)
        self.lin2 = Linear(16, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = self.conv3(x, edge_index)
        x = global_mean_pool(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)

In [10]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GIN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4800, Test Acc: 0.5735, Loss: 0.4985
Epoch: 002, Train Acc: 0.6333, Test Acc: 0.6765, Loss: 0.4553
Epoch: 003, Train Acc: 0.6733, Test Acc: 0.6912, Loss: 0.9511
Epoch: 004, Train Acc: 0.6333, Test Acc: 0.6765, Loss: 0.3461
Epoch: 005, Train Acc: 0.5733, Test Acc: 0.5735, Loss: 0.3202
Epoch: 006, Train Acc: 0.5800, Test Acc: 0.5294, Loss: 0.2515
Epoch: 007, Train Acc: 0.6467, Test Acc: 0.6765, Loss: 0.1860
Epoch: 008, Train Acc: 0.7533, Test Acc: 0.7500, Loss: 0.2593
Epoch: 009, Train Acc: 0.7533, Test Acc: 0.6765, Loss: 0.3195
Epoch: 010, Train Acc: 0.5200, Test Acc: 0.4412, Loss: 0.3335
Epoch: 011, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.2077
Epoch: 012, Train Acc: 0.5200, Test Acc: 0.4412, Loss: 0.3145
Epoch: 013, Train Acc: 0.5000, Test Acc: 0.5882, Loss: 0.1041
Epoch: 014, Train Acc: 0.4933, Test Acc: 0.5882, Loss: 0.1596
Epoch: 015, Train Acc: 0.3467, Test Acc: 0.3529, Loss: 0.3034
Epoch: 016, Train Acc: 0.7000, Test Acc: 0.6912, Loss: 0.2625
Epoch: 0

In [11]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.data import Batch
from torch_geometric.nn import (
    GraphConv,
    JumpingKnowledge,
    global_mean_pool,
    graclus,
    max_pool,
)

num_layers  = 4
hidden = 16
class Graclus(torch.nn.Module):
    def __init__(self):
        super(Graclus, self).__init__()
        self.conv1 = GraphConv(1, 16, aggr='mean')
        self.convs = torch.nn.ModuleList()
        for i in range(num_layers - 1):
            self.convs.append(GraphConv(16, 16, aggr='mean'))
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(64, 16)
        self.lin2 = Linear(16, 1)

    def forward(self, data):
        x, edge_index, weight , batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                cluster = graclus(edge_index,num_nodes=x.size(0))
                data = Batch(x=x, edge_index=edge_index, batch=batch)
                data = max_pool(cluster, data)
                x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        return x

In [12]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Graclus().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6968
Epoch: 002, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7001
Epoch: 003, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6824
Epoch: 004, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6959
Epoch: 005, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6865
Epoch: 006, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6930
Epoch: 007, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7170
Epoch: 008, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6967
Epoch: 009, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6899
Epoch: 010, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6790
Epoch: 011, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6914
Epoch: 012, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6689
Epoch: 013, Train Acc: 0.7133, Test Acc: 0.6765, Loss: 0.6065
Epoch: 014, Train Acc: 0.8533, Test Acc: 0.8529, Loss: 0.5304
Epoch: 015, Train Acc: 0.8533, Test Acc: 0.8529, Loss: 0.4678
Epoch: 016, Train Acc: 0.8533, Test Acc: 0.8529, Loss: 0.3410
Epoch: 0

In [13]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    GraphConv,
    JumpingKnowledge,
    TopKPooling,
    global_mean_pool,
)


class TopK(torch.nn.Module):
    def __init__(self):
        super(TopK, self).__init__()
        self.ratio = 0.8
        self.num_layers = 3
        self.hidden = 16
        self.conv1 = GraphConv(1, 16, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(16, 16, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend(
            [TopKPooling(16, self.ratio) for i in range((self.num_layers) // 2)])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(48, 16)
        self.lin2 = Linear(16, 1)


    def forward(self, data):
        x, edge_index, weight, batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, _, batch, _, _ = pool(x, edge_index,
                                                     batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        #return F.log_softmax(x, dim=-1)
        return x

In [14]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TopK().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6974
Epoch: 002, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6925
Epoch: 003, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6602
Epoch: 004, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6875
Epoch: 005, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6782
Epoch: 006, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6875
Epoch: 007, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6827
Epoch: 008, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6792
Epoch: 009, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6924
Epoch: 010, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6924
Epoch: 011, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6725
Epoch: 012, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7063
Epoch: 013, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6786
Epoch: 014, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6971
Epoch: 015, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6752
Epoch: 016, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6839
Epoch: 0

In [15]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    GraphConv,
    JumpingKnowledge,
    SAGPooling,
    global_mean_pool,
)


class SAGPool(torch.nn.Module):
    def __init__(self):
        super(SAGPool, self).__init__()
        self.num_layers = 4
        self.hidden = 32
        self.ratio = 0.8
        self.conv1 = GraphConv(1, self.hidden, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(self.hidden, self.hidden, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend(
            [SAGPooling(self.hidden, self.ratio) for i in range((self.num_layers) // 2)])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(self.num_layers * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, weight, batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, _, batch, _, _ = pool(x, edge_index,
                                                     batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)

In [16]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SAGPool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6908
Epoch: 002, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6802
Epoch: 003, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6897
Epoch: 004, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6872
Epoch: 005, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7010
Epoch: 006, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7177
Epoch: 007, Train Acc: 0.6067, Test Acc: 0.5588, Loss: 0.6554
Epoch: 008, Train Acc: 0.6200, Test Acc: 0.5588, Loss: 0.5748
Epoch: 009, Train Acc: 0.8133, Test Acc: 0.8088, Loss: 0.4155
Epoch: 010, Train Acc: 0.7800, Test Acc: 0.7941, Loss: 0.4403
Epoch: 011, Train Acc: 0.8533, Test Acc: 0.8529, Loss: 0.3327
Epoch: 012, Train Acc: 0.8267, Test Acc: 0.8088, Loss: 0.3000
Epoch: 013, Train Acc: 0.8533, Test Acc: 0.8529, Loss: 0.1445
Epoch: 014, Train Acc: 0.8533, Test Acc: 0.8529, Loss: 0.3918
Epoch: 015, Train Acc: 0.8533, Test Acc: 0.8529, Loss: 0.3244
Epoch: 016, Train Acc: 0.8533, Test Acc: 0.8529, Loss: 0.2635
Epoch: 0

In [17]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    EdgePooling,
    GraphConv,
    JumpingKnowledge,
    global_mean_pool,
)


class EdgePool(torch.nn.Module):
    def __init__(self):
        super(EdgePool, self).__init__()
        self.num_layers = 3 
        self.hidden = 16
        self.conv1 = GraphConv(1, self.hidden, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(self.hidden, self.hidden, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend(
            [EdgePooling(self.hidden) for i in range((self.num_layers) // 2)])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(self.num_layers * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, weight ,  batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, batch, _ = pool(x, edge_index, batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [18]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EdgePool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6949
Epoch: 002, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7026
Epoch: 003, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7020
Epoch: 004, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6793
Epoch: 005, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6843
Epoch: 006, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7046
Epoch: 007, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6817
Epoch: 008, Train Acc: 0.5400, Test Acc: 0.4706, Loss: 0.6707
Epoch: 009, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.5453
Epoch: 010, Train Acc: 0.9267, Test Acc: 0.9118, Loss: 0.5704
Epoch: 011, Train Acc: 0.7000, Test Acc: 0.7059, Loss: 0.4496
Epoch: 012, Train Acc: 0.8667, Test Acc: 0.8971, Loss: 0.4826
Epoch: 013, Train Acc: 0.7600, Test Acc: 0.7500, Loss: 0.4557
Epoch: 014, Train Acc: 0.9533, Test Acc: 0.9412, Loss: 0.2612
Epoch: 015, Train Acc: 0.7533, Test Acc: 0.7647, Loss: 0.3709
Epoch: 016, Train Acc: 0.9267, Test Acc: 0.9265, Loss: 0.1545
Epoch: 0

In [19]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import GlobalAttention, SAGEConv


class GlobalAttentionNet(torch.nn.Module):
    def __init__(self):
        super(GlobalAttentionNet, self).__init__()
        self.num_layers = 3
        self.hidden  = 16
        self.conv1 = SAGEConv(1, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(self.hidden, self.hidden))
        self.att = GlobalAttention(Linear(self.hidden, 1))
        self.lin1 = Linear(self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = self.att(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)


In [20]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GlobalAttentionNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7032


/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'nn.glob.GlobalAttention' is deprecated, use 'nn.aggr.AttentionalAggregation' instead
  warnings.warn(out)


Epoch: 002, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6945
Epoch: 003, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6949
Epoch: 004, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6862
Epoch: 005, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7000
Epoch: 006, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6864
Epoch: 007, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6696
Epoch: 008, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6856
Epoch: 009, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6506
Epoch: 010, Train Acc: 0.6000, Test Acc: 0.5588, Loss: 0.5713
Epoch: 011, Train Acc: 0.8533, Test Acc: 0.8529, Loss: 0.6562
Epoch: 012, Train Acc: 0.8533, Test Acc: 0.8529, Loss: 0.3625
Epoch: 013, Train Acc: 0.8533, Test Acc: 0.8529, Loss: 0.5390
Epoch: 014, Train Acc: 0.8533, Test Acc: 0.8529, Loss: 0.5301
Epoch: 015, Train Acc: 0.8533, Test Acc: 0.8529, Loss: 0.4490
Epoch: 016, Train Acc: 0.8533, Test Acc: 0.8529, Loss: 0.4466
Epoch: 017, Train Acc: 0.8533, Test Acc: 0.8529, Loss: 0.4619
Epoch: 0

In [21]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import SAGEConv, Set2Set


class Set2SetNet(torch.nn.Module):
    def __init__(self):
        super(Set2SetNet,self).__init__()
        self.hidden = 16
        self.num_layers = 3 
        self.conv1 = SAGEConv(1, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(self.hidden, self.hidden))
        self.set2set = Set2Set(self.hidden, processing_steps=4)
        self.lin1 = Linear(2 * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = self.set2set(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [22]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Set2SetNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'nn.glob.Set2Set' is deprecated, use 'nn.aggr.Set2Set' instead
  warnings.warn(out)


Epoch: 001, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6998
Epoch: 002, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6959
Epoch: 003, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.7060
Epoch: 004, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.7054
Epoch: 005, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6924
Epoch: 006, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6895
Epoch: 007, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.7008
Epoch: 008, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6845
Epoch: 009, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.7027
Epoch: 010, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6973
Epoch: 011, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6971
Epoch: 012, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.7013
Epoch: 013, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.7051
Epoch: 014, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6965
Epoch: 015, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6888
Epoch: 016, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6960
Epoch: 0

In [23]:
import torch
import torch.nn.functional as F
from torch.nn import Conv1d, Linear

from torch_geometric.nn import SAGEConv, global_sort_pool


class SortPool(torch.nn.Module):
    def __init__(self):
        super(SortPool, self).__init__()
        self.k = 30
        self.num_layers = 3 
        self.hidden = 16
        self.conv1 = SAGEConv(1, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(self.hidden, self.hidden))
        self.conv1d = Conv1d(self.hidden, 32, 5)
        self.lin1 = Linear(32 * (self.k - 5 + 1), self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = global_sort_pool(x, batch, self.k)
        x = x.view(len(x), self.k, -1).permute(0, 2, 1)
        x = F.relu(self.conv1d(x))
        x = x.view(len(x), -1)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [24]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SortPool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 200):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'nn.glob.global_sort_pool' is deprecated, use 'nn.aggr.SortAggr' instead
  warnings.warn(out)


Epoch: 001, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6714
Epoch: 002, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6936
Epoch: 003, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6865
Epoch: 004, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6998
Epoch: 005, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6743
Epoch: 006, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.7105
Epoch: 007, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6703
Epoch: 008, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.7084
Epoch: 009, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6974
Epoch: 010, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6777
Epoch: 011, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.7141
Epoch: 012, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6995
Epoch: 013, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.7088
Epoch: 014, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6973
Epoch: 015, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6866
Epoch: 016, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.6915
Epoch: 0

In [25]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    ASAPooling,
    GraphConv,
    JumpingKnowledge,
    global_mean_pool,
)


class ASAP(torch.nn.Module):
    def __init__(self):
        super(ASAP, self).__init__()
        self.num_layers = 5 
        self.hidden  = 32
        self.ratio = 0.8
        self.dropout = 0.2
        self.conv1 = GraphConv(1, self.hidden, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(self.hidden, self.hidden, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend([
            ASAPooling(self.hidden, self.ratio, dropout=self.dropout)
            for i in range((self.num_layers) // 2)
        ])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(self.num_layers * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index,data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = conv(x=x, edge_index=edge_index, edge_weight=edge_weight)
            x = F.relu(x)
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, edge_weight, batch, _ = pool(
                    x=x, edge_index=edge_index, edge_weight=edge_weight,
                    batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)


In [26]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ASAP().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4733, Test Acc: 0.5588, Loss: 0.7004
Epoch: 002, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6932
Epoch: 003, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7033
Epoch: 004, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7014
Epoch: 005, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6808
Epoch: 006, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6921
Epoch: 007, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6836
Epoch: 008, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6918
Epoch: 009, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7005
Epoch: 010, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6740
Epoch: 011, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7107
Epoch: 012, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6822
Epoch: 013, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6795
Epoch: 014, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.7285
Epoch: 015, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6774
Epoch: 016, Train Acc: 0.5267, Test Acc: 0.4412, Loss: 0.6889
Epoch: 0